# Zillow Web Scrapping

In this approach, we fetch all url from the property list and then fetch each web page data to extract get info explictly

Advantages: 
   - We can populate the data with lot of info
   
Disadvantages: 
  - Multiple time scrapping results in CAPTCHA page that is implemented by zillow to ensure non-stopping scrapes.
  - The property list property, we are able to collect few property info at a time.
  - Lot of time to scrape.

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import csv


# Setting up different header settings

req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}


def scrapeUrl(page_link):
    try:
        with requests.Session() as s:
            page_response = s.get(page_link,timeout=5,headers=req_headers)
            if page_response.status_code == 200:
                return page_response.text
            else:
                print(page_link,page_response.status_code)
    except requests.Timeout as e:
        print(str(e))
    except:
        print("Some error in url",page_link)
    return ''    

# Generate list of all urls in a zip code
def create_url(zipcode):
    #url = "https://www.zillow.com/homes/for_sale/{0}/0_singlestory/days_sort".format(zipcode)
    url="https://www.zillow.com/homes/{0}_rb/".format(zipcode)
    parser = scrapeUrl(url)
    soup = BeautifulSoup(parser, 'html.parser')
    arr=soup.find_all('a',{"class":"list-card-link"},href=True)

    urls_list=[]
    for s in arr:
        if str(s):
            interim_url = (str(s).split('href=')[1])
            if interim_url:
                url=(interim_url.split('tabindex')[0]).replace('"','')
                urls_list.append(url.strip())
    return urls_list

#Generate property list
def property_type(property):
    property_list=['apartment','single-family','townhouse','condo']
    for property_type in property_list:
        if property_type in property:
            return property_type
        
    return ''
            
# Extract info for each property
def extractPropertyInfo(property_url):
    html = scrapeUrl(property_url)
    if html!='':
        soup = BeautifulSoup(html, 'html.parser')
        title = soup.find('title')
        url_title=str(title).replace('title>','').replace("<","").replace("| Zillow",'')
        if len(url_title.split(','))>0:
            url_address = url_title.split(',')[0]
        else:
            url_address=''
        if len(url_title.split(','))>=2:
            url_city = url_title.split(',')[1]
        else:
            url_city=''
            
        if len(url_title.split(','))==3 and url_title.split(',')[2]:
            url_state = re.findall(r'([a-zA-Z]+)', url_title.split(',')[2])[0]
            url_postal_code = re.findall(r'([0-9]+)', url_title.split(',')[2])[0]
        else:
            url_state = ''
            url_postal_code = ''
        meta_tag = soup.find('meta', attrs={'name': 'description'})
        price_list = re.findall(r'\$([0-9]+,*[0-9]+,*[0-9]+)', str(meta_tag))
        if len(price_list)>0:
            url_price=price_list[0]
        else:
            url_price=None
            
        ft_list = re.findall(r'([0-9]+,*[0-9]+)sq.ft', str(meta_tag).replace(' ',''))
        if len(ft_list)>0:
            url_ft=ft_list[0]
        else:
            url_ft=0
            
        beds_list = re.findall(r'([0-9]+\.*[0-9]*)bed', str(meta_tag).replace(' ','' ))  
        if len(beds_list)>0:
            url_beds = beds_list[0]
        else:
            url_beds=0
            
        bath_list = re.findall(r'([0-9]+\.*[0-9]*)bath', str(meta_tag).replace(' ','' ))
        if len(bath_list)>0:
            url_bath=bath_list[0]
        else:
            url_bath=0
        
        url_house_type = property_type(str(meta_tag))  
            
        properties = {'address': url_address,
                      'city': url_city,
                      'state': url_state,
                      'postal_code': url_postal_code,
                      'price': url_price,
                      'size': url_ft,
                      'bed': url_beds,
                      'bath':url_bath,
                      'house_type':url_house_type,
                      'url': property_url,
                      'title': url_title
                      }
        return properties
    return ''

#Scrape the url and fetch content data     
def fetchPropertyInfo(zipcode):
    url_list = create_url(zipcode)
    property_list_info=[]
    for l in url_list:
        property_dict = extractPropertyInfo(l)
        if property_dict!='' and property_dict['price']:
            property_list_info.append(property_dict)
    
    return  property_list_info

#Write the code to csv file        
def write_data_to_csv(data,zipcode):
    with open("properties-%s.csv" % (zipcode), 'w') as csvfile:
        fieldnames = ['title', 'address', 'city', 'state', 'postal_code', 'price', 'size', 'bed', 'bath','house_type','url']
        dict_writer = csv.DictWriter(csvfile, fieldnames)
        dict_writer.writeheader()
        dict_writer.writerows(data)
        print("File written", "properties-%s.csv" % (zipcode))

if __name__ == "__main__":
    all_property_list=fetchPropertyInfo(94536)
    write_data_to_csv(all_property_list,94536)       


File written properties-94536.csv


We use paginated approach, fetch all info from property listing page and introduce sleep between each page scrapped.

Advantages: 
   - Flexibility to extract all pages or specific pages with only url changes.
   - Reduce the number of hits to zillow page 
   - Eliminate CAPTCHA page from zillow (as sleep time introduced).


In [3]:
import urllib.request 
import requests
import urllib.parse
import urllib.error
import ssl
import time
import json
import csv
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as BS


# For ignoring SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

class zillow_scrape():
    #Parameter for property list
    results=[]
    
    #Headers info working 
    headers={
   'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
   'accept-encoding': 'gzip, deflate, br',
   'accept-language': 'en-US,en;q=0.9,hi;q=0.8',
   'cache-control': 'max-age=0',
   'cookie': 'zguid=23|%24eb9b2468-daec-4f5b-8685-9e5054e18c61; zgsession=1|db619838-8377-49d1-9bc2-f79a09b10290; _ga=GA1.2.522299916.1600303789; _gid=GA1.2.330266850.1600303789 zjs_user_id=null; zjs_anonymous_id=%22eb9b2468-daec-4f5b-8685-9e5054e18c61%22; _pxvid=b14a55b1-f87f-11ea-90f4-0242ac120007; FSsampler=2047642450; _gcl_au=1.1.2041567322.1600303795; KruxPixel=true; DoubleClickSession=true; _fbp=fb.1.1600303795829.1293531853; ki_r=; _pin_unauth=dWlkPU56RmtNbVEzTVdJdE1XUm1aUzAwT1dVNUxUZ3haalF0TURVNU9EZzBabUppTVRKaSZycD1kSEoxWlE; __gads=ID=bb6eca7673411b1f:T=1600303796:S=ALNI_MaUgjH6XKP_LrNZWI8dWwnc1dWxIQ; ki_s=; g_state={"i_p":1600311000745,"i_l":1}; KruxAddition=true; JSESSIONID=7561BA38C0EBB35E84FAAE495486C7CD; search=6|1602898003029%7Cregion%3Dfremont-ca%26rect%3D38.35%252C-120.98%252C36.35%252C-122.98%26disp%3Dmap%26mdm%3Dauto%26sort%3Ddays%26fs%3D0%26fr%3D1%26mmm%3D0%26rs%3D0%26ah%3D0%09%0911540%09%09%09%09%09%09; _px3=1b6fd8f975b090edf69c79b567074785a0d78c40c5df40b89959662dc9684142:DL7CCxp+jR7ow5BgB1kNmhAt9YUfi82UjC26E8MhI8DUEyZz5tj00bXJvJkWxCCmkt91/1n8MqgnsMK3HB8zOw==:1000:iBOOFI5BZmbPT6QuRyCSEILwqxoObf+XjBjn/f7i+4T2z7onbf3tlIO4i2Q5BiNGfCpF0QmJCntZPwfciCIi4Lv8SmRhoP+5ZAwIzo64IM/YC+58YwvIZoXkAtF5FERk4L4sCfKT5eWHJw8IBtPfubEDMA3e9VFCvg/UKtzuegs=; AWSALB=4zzwyr6pR9IACkbg5DfvzJ3ouS/3Eb6sIJlJdD+8+dhSyWfYpuKAz6DGJpbFwp+pOS064dTBVINf2c95CwC1scfjERQPAJLkqwIb3UqDYCnK76iksbzvz/NaA1Ea; AWSALBCORS=4zzwyr6pR9IACkbg5DfvzJ3ouS/3Eb6sIJlJdD+8+dhSyWfYpuKAz6DGJpbFwp+pOS064dTBVINf2c95CwC1scfjERQPAJLkqwIb3UqDYCnK76iksbzvz/NaA1Ea; _uetsid=cadea8b803e050be667596131d786d51; _uetvid=75de6c04fd74c5089035015a0bd81951; ki_t=1600303796100%3B1600303796100%3B1600306010341%3B1%3B11; _derived_epik=dj0yJnU9Z1hJOTU1ZG53ekJlaFVtUlcxa2lkc3RtLW1tM0F1UWwmbj1VSTVfTl9lUnNTaHBfMkJ1cmFaeXJnJm09MSZ0PUFBQUFBRjlpdTFzJnJtPTEmcnQ9QUFBQUFGOWl1MXM',
   'sec-fetch-dest': 'document',
   'sec-fetch-mode': 'navigate',
   'sec-fetch-site': 'same-origin',
   'sec-fetch-user': '?1',
   'upgrade-insecure-requests': '1',
   'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36',
    }
    
    #Fetch the page 
    def fetch(self,url,params):
        response=requests.get(url,headers=self.headers,params=params)
        #req = Request(url, headers=self.headers,params=params)
        #response = urlopen(req).read()
        print(response.status_code)
        return response
   
    #Parse each page sequencially
    def parse(self,response):
        content=BS(response,'html.parser')
        elements=content.find('ul',{'class':"photo-cards photo-cards_wow photo-cards_short"})
        
        for x in elements:
            script= x.find('script',{'type':"application/ld+json"})
            if script:
                script_json = json.loads(script.contents[0])
                
                if script_json['@type']=='SingleFamilyResidence':
                    if  x.find('div',{'class':'list-card-variable-text list-card-img-overlay'}):
                        no_of_days =  x.find('div',{'class':'list-card-variable-text list-card-img-overlay'}).text
                    else:
                        no_of_days= None
                    
                    
                    property_dict={'name': script_json['name'],
                               'street':script_json['address']['streetAddress'],
                               'city': script_json['address']['addressLocality'],
                               'state':script_json['address']['addressRegion'],
                               'postalCode':script_json['address']['postalCode'],
                               'floorSize': script_json['floorSize']['value'],
                               'price': x.find('div', {'class': 'list-card-price'}).text,
                               'type_of_house': x.find('ul', {'class': 'list-card-details'}).text,
                               'property_type':  x.find('div', {'class': 'list-card-type'}).text,
                               'no_of_days': no_of_days,
                               'url':script_json['url']
                            }
                    if property_dict!='' and property_dict['price']: 
                        self.results.append(property_dict)
    
    #Method to store the csv file
    def to_csv(self):
        filename='zillow_data.csv'
        with open(filename,'w',newline='') as csv_file:
            fieldnames = ['name', 'street', 'city', 'state', 'postalCode', 'floorSize', 'price', 'type_of_house', 'property_type','no_of_days','url']
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames,delimiter=',')
            writer.writeheader()
            for row in self.results:
                    writer.writerow(row)
        print("CSV file written to file : ", filename)
                    
    # Fetch the list of the urls
    def fetchData(self):
        # Specify first few and last few pages 
        urls=['https://www.zillow.com/fremont-ca/rentals/?searchQueryState=%7B%22mapBounds%22%3A%7B%22west%22%3A-122.20931431103516%2C%22east%22%3A-121.80762668896485%2C%22south%22%3A37.41580104821932%2C%22north%22%3A37.64287323941954%7D%2C%22mapZoom%22%3A12%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A11540%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D'
              ,'https://www.zillow.com/fremont-ca/rentals/2_p/?searchQueryState=%7B%22mapBounds%22%3A%7B%22west%22%3A-122.20931431103516%2C%22east%22%3A-121.80762668896485%2C%22south%22%3A37.41580104821932%2C%22north%22%3A37.64287323941954%7D%2C%22mapZoom%22%3A12%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A11540%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22pagination%22%3A%7B%22currentPage%22%3A2%7D%7D'
              ,'https://www.zillow.com/fremont-ca/rentals/7_p/?searchQueryState=%7B%22mapBounds%22%3A%7B%22west%22%3A-122.20931431103516%2C%22east%22%3A-121.80762668896485%2C%22south%22%3A37.41580104821932%2C%22north%22%3A37.64287323941954%7D%2C%22mapZoom%22%3A12%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A11540%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22pagination%22%3A%7B%22currentPage%22%3A7%7D%7D'
              ,'https://www.zillow.com/fremont-ca/rentals/6_p/?searchQueryState=%7B%22mapBounds%22%3A%7B%22west%22%3A-122.20931431103516%2C%22east%22%3A-121.80762668896485%2C%22south%22%3A37.41580104821932%2C%22north%22%3A37.64287323941954%7D%2C%22mapZoom%22%3A12%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A11540%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22pagination%22%3A%7B%22currentPage%22%3A6%7D%7D'
              ,'https://www.zillow.com/santa-clara-ca/rentals/8_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A8%7D%2C%22usersSearchTerm%22%3A%22Santa%20Clara%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-122.07385590820311%2C%22east%22%3A-121.86236909179686%2C%22south%22%3A37.29965215571662%2C%22north%22%3A37.442078009553%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A13713%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22doz%22%3A%7B%22value%22%3A%2236m%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A12%7D'
              ,'https://www.zillow.com/santa-clara-ca/rentals/7_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A7%7D%2C%22usersSearchTerm%22%3A%22Santa%20Clara%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-122.16895631103515%2C%22east%22%3A-121.76726868896483%2C%22south%22%3A37.25703564678547%2C%22north%22%3A37.48458955926386%7D%2C%22mapZoom%22%3A12%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A13713%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22doz%22%3A%7B%22value%22%3A%2236m%22%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D'
              ,'https://www.zillow.com/santa-clara-ca/rentals/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Santa%20Clara%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-122.16895631103515%2C%22east%22%3A-121.76726868896483%2C%22south%22%3A37.25703564678547%2C%22north%22%3A37.48458955926386%7D%2C%22mapZoom%22%3A12%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A13713%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22doz%22%3A%7B%22value%22%3A%2236m%22%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D'
              ,'https://www.zillow.com/santa-clara-ca/rentals/2_p/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Santa%20Clara%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-122.16895631103515%2C%22east%22%3A-121.76726868896483%2C%22south%22%3A37.25703564678547%2C%22north%22%3A37.48458955926386%7D%2C%22mapZoom%22%3A12%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A13713%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22doz%22%3A%7B%22value%22%3A%2236m%22%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22pagination%22%3A%7B%22currentPage%22%3A2%7D%7D'
              ,'https://www.zillow.com/sunnyvale-ca/rentals/11_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A11%7D%2C%22usersSearchTerm%22%3A%22Sunnyvale%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-122.12971090820311%2C%22east%22%3A-121.91822409179686%2C%22south%22%3A37.3182197415321%2C%22north%22%3A37.46061037010319%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A54626%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22doz%22%3A%7B%22value%22%3A%2236m%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A12%7D'
              ,'https://www.zillow.com/sunnyvale-ca/rentals/10_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A10%7D%2C%22usersSearchTerm%22%3A%22Sunnyvale%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-122.22481131103514%2C%22east%22%3A-121.82312368896483%2C%22south%22%3A37.275613750383386%2C%22north%22%3A37.50311138351423%7D%2C%22mapZoom%22%3A12%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A54626%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22doz%22%3A%7B%22value%22%3A%2236m%22%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D'
              ,'https://www.zillow.com/sunnyvale-ca/rentals/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Sunnyvale%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-122.22481131103514%2C%22east%22%3A-121.82312368896483%2C%22south%22%3A37.275613750383386%2C%22north%22%3A37.50311138351423%7D%2C%22mapZoom%22%3A12%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A54626%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22doz%22%3A%7B%22value%22%3A%2236m%22%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D'
              ,'https://www.zillow.com/sunnyvale-ca/rentals/2_p/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Sunnyvale%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-122.22481131103514%2C%22east%22%3A-121.82312368896483%2C%22south%22%3A37.275613750383386%2C%22north%22%3A37.50311138351423%7D%2C%22mapZoom%22%3A12%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A54626%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22doz%22%3A%7B%22value%22%3A%2236m%22%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22pagination%22%3A%7B%22currentPage%22%3A2%7D%7D'
              ,'https://www.zillow.com/mountain-view-ca/rentals/12_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A12%7D%2C%22usersSearchTerm%22%3A%22Mountain%20View%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-122.18701040820312%2C%22east%22%3A-121.97552359179687%2C%22south%22%3A37.33249175379281%2C%22north%22%3A37.474855296256855%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A32999%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22doz%22%3A%7B%22value%22%3A%2236m%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A12%7D'
              ,'https://www.zillow.com/mountain-view-ca/rentals/11_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A11%7D%2C%22usersSearchTerm%22%3A%22Mountain%20View%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-122.28211081103515%2C%22east%22%3A-121.88042318896484%2C%22south%22%3A37.28989385020283%2C%22north%22%3A37.51734820790566%7D%2C%22mapZoom%22%3A12%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A32999%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22doz%22%3A%7B%22value%22%3A%2236m%22%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D'
              ,'https://www.zillow.com/mountain-view-ca/rentals/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Mountain%20View%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-122.28211081103515%2C%22east%22%3A-121.88042318896484%2C%22south%22%3A37.28989385020283%2C%22north%22%3A37.51734820790566%7D%2C%22mapZoom%22%3A12%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A32999%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22doz%22%3A%7B%22value%22%3A%2236m%22%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D'
              ,'https://www.zillow.com/mountain-view-ca/rentals/2_p/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Mountain%20View%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-122.28211081103515%2C%22east%22%3A-121.88042318896484%2C%22south%22%3A37.28989385020283%2C%22north%22%3A37.51734820790566%7D%2C%22mapZoom%22%3A12%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A32999%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22doz%22%3A%7B%22value%22%3A%2236m%22%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22pagination%22%3A%7B%22currentPage%22%3A2%7D%7D'
              ,'https://www.zillow.com/palo-alto-ca/rentals/9_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A9%7D%2C%22usersSearchTerm%22%3A%22Palo%20Alto%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-122.33651481103517%2C%22east%22%3A-121.93482718896486%2C%22south%22%3A37.267806832424895%2C%22north%22%3A37.49532811824657%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A26374%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22doz%22%3A%7B%22value%22%3A%2236m%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A12%7D'
              ,'https://www.zillow.com/palo-alto-ca/rentals/8_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A8%7D%2C%22usersSearchTerm%22%3A%22Palo%20Alto%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-122.33651481103517%2C%22east%22%3A-121.93482718896486%2C%22south%22%3A37.267806832424895%2C%22north%22%3A37.49532811824657%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A26374%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22doz%22%3A%7B%22value%22%3A%2236m%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A12%7D'
              ,'https://www.zillow.com/palo-alto-ca/rentals/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Palo%20Alto%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-122.33651481103517%2C%22east%22%3A-121.93482718896486%2C%22south%22%3A37.267806832424895%2C%22north%22%3A37.49532811824657%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A26374%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22doz%22%3A%7B%22value%22%3A%2236m%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A12%7D'
              ,'https://www.zillow.com/palo-alto-ca/rentals/2_p/?searchQueryState=%7B%22usersSearchTerm%22%3A%22Palo%20Alto%2C%20CA%22%2C%22mapBounds%22%3A%7B%22west%22%3A-122.33651481103517%2C%22east%22%3A-121.93482718896486%2C%22south%22%3A37.267806832424895%2C%22north%22%3A37.49532811824657%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A26374%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22doz%22%3A%7B%22value%22%3A%2236m%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A12%2C%22pagination%22%3A%7B%22currentPage%22%3A2%7D%7D'              
             ]
          
        for url in urls:
            #Fetch few pages for number of days in zillow
            for page in range(1):
                params= {
                    'searchQueryState': '{"pagination":{},"mapBounds":{"west":-122.37239261914 063,"east":-121.64454838085938,"north":37.83048686494093,"south":37.22714015323799},"regionSelection":[{"regionId":11540,"regionType":6}],"isMapVisible":false,"filterState":{"pmf":{"value":false},"fore":{"value":false},"mf":{"value":false},"auc":{"value":false},"nc":{"value":false},"fr":{"value":true},"land":{"value":false},"manu":{"value":false},"fsbo":{"value":false},"cmsn":{"value":false},"pf":{"value":false},"fsba":{"value":false}},"isListVisible":true}'
                }
                result=self.fetch(url,params)
                self.parse(result.text)
                #Place a sleep time
                time.sleep(2)
        self.to_csv() 
        
                
if __name__ == "__main__":
    scrapper= zillow_scrape()
    scrapper.fetchData()

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
CSV file written to file :  zillow_data.csv
